<a href="https://colab.research.google.com/github/rushikeshnakhate/PortfoliOpt/blob/master/multiple_dates_expected_return_df_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# data_dir = Path("/content/drive/MyDrive/data")
# !pip install tqdm
# !pip install PyPortfolioOpt
# !pip install tabulate
# !pip install dtale
# !pip install -q data-table
# from google.colab import data_table

In [24]:
import os
from pathlib import Path
import pandas as pd
from tabulate import tabulate

data_dir = Path(r"D:\PortfoliOpt\data")

In [25]:
def load_all_pickles_to_dfs(directory):
    """
    Reads all .pkl files in the specified directory and loads each file into a DataFrame.
    
    Parameters:
    - directory (str): The path to the directory containing .pkl files.

    Returns:
    - dict: A dictionary where keys are file names without extensions and values are DataFrames.
    """
    dataframes = {}
    for filename in os.listdir(directory):
        if filename.endswith(".pkl"):
            file_path = os.path.join(directory, filename)
            try:
                df = pd.read_pickle(file_path)
                dataframes[filename[:-4]] = df  # Using filename without .pkl extension as the key
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    return dataframes

In [26]:
from datetime import datetime, timedelta


def generate_month_date_ranges(year, months=None):
    if months is None:
        months = range(1, 13)  # Default to all months if none specified

    month_ranges = []
    for month in months:
        start_date = datetime(year, month, 1).date()
        if month == 12:
            end_date = (datetime(year + 1, 1, 1) - timedelta(days=1)).date()
        else:
            end_date = (datetime(year, month + 1, 1) - timedelta(days=1)).date()
        month_ranges.append((start_date, end_date))
    return month_ranges


def create_current_month_directory(start_date):
    current_month = start_date.strftime("%Y%m")
    current_month_dir = data_dir / current_month
    return current_month_dir

In [30]:
if __name__ == "__main__":
    year = 2023
    alternate_months = [1]
    month_ranges = generate_month_date_ranges(year, months=alternate_months)

    for start_date, end_date in month_ranges:
        all_pkl_file_dir = create_current_month_directory(start_date=start_date)
        all_dfs = load_all_pickles_to_dfs(all_pkl_file_dir)
        for file_name, df in all_dfs.items():
            print(f"DataFrame from {file_name}.pkl")
            # print(df)
            print(tabulate(df, headers='keys', tablefmt='pretty'))

DataFrame from covariance_CopulaRiskModel.pkl
+-------------+------------------------+-------------------------+
|   Ticker    |      HDFCBANK.NS       |       RELIANCE.NS       |
+-------------+------------------------+-------------------------+
| HDFCBANK.NS | 4.2683766775904385e-05 | -6.1483137326696985e-06 |
| RELIANCE.NS | -6.148313732669699e-06 |  6.982669371227165e-05  |
+-------------+------------------------+-------------------------+
DataFrame from covariance_ExponentialCovariance.pkl
+-------------+----------------------+----------------------+
|   Ticker    |     HDFCBANK.NS      |     RELIANCE.NS      |
+-------------+----------------------+----------------------+
| HDFCBANK.NS | 0.042362843188272055 | 0.013217124135309469 |
| RELIANCE.NS | 0.013217124135309469 | 0.03304031417183516  |
+-------------+----------------------+----------------------+
DataFrame from covariance_GaussianProcessRiskModel.pkl
+-------------+------------------------+------------------------+
|   Tic

In [29]:
df = pd.read_pickle(
    r"D:\PortfoliOpt\data\202301\optimization_GordonGrowthReturn_SVMVolatility_PyPortfolioOptFrontierWithShortPosition.pkl")
print(df)

                            Cleaned Weights  Expected Annual Return  \
0  {'HDFCBANK.NS': 1.0, 'RELIANCE.NS': 0.0}                  0.4092   

   Annual Volatility  Sharpe Ratio  
0           0.111837      3.480072  
